# Phân tích Cơ sở dữ liệu SQL Server - Bài 1: Kết nối và Phân tích Cơ bản

Notebook này hướng dẫn cách kết nối đến SQL Server và thực hiện các phân tích cơ bản.

## 🔧 Bước 1: Import thư viện và thiết lập môi trường

In [ ]:
# Import các thư viện cần thiết
import sys
sys.path.append('/app')

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Import database helper
from db_connection import get_db_connection

# Thiết lập style cho biểu đồ
plt.style.use('default')
sns.set_palette("husl")
%matplotlib inline

# Thiết lập hiển thị pandas
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("✅ Đã import thành công tất cả thư viện!")
print(f"📅 Thời gian chạy: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

## 🔌 Bước 2: Kết nối cơ sở dữ liệu

In [ ]:
# Kết nối đến SQL Server
try:
    db = get_db_connection()
    print("✅ Đã kết nối thành công đến SQL Server!")
    
    # Kiểm tra kết nối
    test_query = "SELECT @@VERSION as SQLServerVersion"
    version_info = db.query_to_dataframe(test_query)
    print(f"📊 Phiên bản SQL Server: {version_info.iloc[0]['SQLServerVersion'][:50]}...")
    
except Exception as e:
    print(f"❌ Lỗi kết nối: {e}")
    raise

## 📋 Bước 3: Khám phá cấu trúc cơ sở dữ liệu

In [ ]:
# Xem danh sách bảng
tables = db.get_table_info()
print("📋 Danh sách bảng trong database ShopDB:")
display(tables)

print(f"\n📊 Tổng số bảng: {len(tables)}")

In [ ]:
# Xem cấu trúc từng bảng
table_names = ['Customers', 'Products', 'Orders', 'OrderDetails']

for table_name in table_names:
    print(f"\n🔍 Cấu trúc bảng {table_name}:")
    table_structure = db.get_table_info(table_name)
    display(table_structure)

## 📊 Bước 4: Tải và khám phá dữ liệu

In [ ]:
# Tải dữ liệu từ tất cả các bảng
print("📥 Đang tải dữ liệu từ các bảng...")

customers = db.query_to_dataframe("SELECT * FROM Customers")
products = db.query_to_dataframe("SELECT * FROM Products")
orders = db.query_to_dataframe("SELECT * FROM Orders")
order_details = db.query_to_dataframe("SELECT * FROM OrderDetails")

print("\n📊 Thông tin tổng quan về dữ liệu:")
print(f"👥 Khách hàng: {len(customers)} records")
print(f"🛍️ Sản phẩm: {len(products)} records")
print(f"📦 Đơn hàng: {len(orders)} records")
print(f"📋 Chi tiết đơn hàng: {len(order_details)} records")

print("\n✅ Đã tải thành công tất cả dữ liệu!")

In [ ]:
# Hiển thị mẫu dữ liệu từ mỗi bảng
print("👥 Mẫu dữ liệu bảng Customers:")
display(customers.head())
print(f"Thông tin: {customers.shape[0]} khách hàng, {customers.shape[1]} cột")

print("\n🛍️ Mẫu dữ liệu bảng Products:")
display(products.head())
print(f"Thông tin: {products.shape[0]} sản phẩm, {products.shape[1]} cột")

In [ ]:
print("📦 Mẫu dữ liệu bảng Orders:")
display(orders.head())
print(f"Thông tin: {orders.shape[0]} đơn hàng, {orders.shape[1]} cột")

print("\n📋 Mẫu dữ liệu bảng OrderDetails:")
display(order_details.head())
print(f"Thông tin: {order_details.shape[0]} chi tiết đơn hàng, {order_details.shape[1]} cột")

## 📈 Bước 5: Phân tích cơ bản

In [ ]:
# Thống kê cơ bản về khách hàng
print("👥 PHÂN TÍCH KHÁCH HÀNG")
print("=" * 50)

print(f"📊 Tổng số khách hàng: {len(customers)}")
print(f"🏙️ Số thành phố: {customers['City'].nunique()}")
print("\n🏙️ Phân bố khách hàng theo thành phố:")
city_distribution = customers['City'].value_counts()
display(city_distribution)

# Biểu đồ phân bố khách hàng theo thành phố
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
city_distribution.plot(kind='bar', color='skyblue')
plt.title('Số lượng khách hàng theo thành phố')
plt.xlabel('Thành phố')
plt.ylabel('Số lượng khách hàng')
plt.xticks(rotation=45)

plt.subplot(1, 2, 2)
plt.pie(city_distribution.values, labels=city_distribution.index, autopct='%1.1f%%')
plt.title('Tỷ lệ khách hàng theo thành phố')

plt.tight_layout()
plt.show()

In [ ]:
# Thống kê cơ bản về sản phẩm
print("🛍️ PHÂN TÍCH SẢN PHẨM")
print("=" * 50)

print(f"📊 Tổng số sản phẩm: {len(products)}")
print(f"📂 Số danh mục: {products['Category'].nunique()}")
print(f"💰 Giá trung bình: ${products['Price'].mean():.2f}")
print(f"💰 Giá cao nhất: ${products['Price'].max():.2f}")
print(f"💰 Giá thấp nhất: ${products['Price'].min():.2f}")

print("\n📂 Phân bố sản phẩm theo danh mục:")
category_distribution = products['Category'].value_counts()
display(category_distribution)

# Biểu đồ phân tích sản phẩm
plt.figure(figsize=(15, 10))

# Phân bố theo danh mục
plt.subplot(2, 2, 1)
category_distribution.plot(kind='bar', color='lightgreen')
plt.title('Số lượng sản phẩm theo danh mục')
plt.xlabel('Danh mục')
plt.ylabel('Số lượng sản phẩm')
plt.xticks(rotation=45)

# Phân bố giá
plt.subplot(2, 2, 2)
plt.hist(products['Price'], bins=10, color='orange', alpha=0.7)
plt.title('Phân bố giá sản phẩm')
plt.xlabel('Giá ($)')
plt.ylabel('Số lượng sản phẩm')

# Giá theo danh mục
plt.subplot(2, 2, 3)
sns.boxplot(data=products, x='Category', y='Price')
plt.title('Phân bố giá theo danh mục')
plt.xticks(rotation=45)

# Top sản phẩm đắt nhất
plt.subplot(2, 2, 4)
top_expensive = products.nlargest(5, 'Price')
plt.barh(top_expensive['ProductName'], top_expensive['Price'], color='red', alpha=0.7)
plt.title('Top 5 sản phẩm đắt nhất')
plt.xlabel('Giá ($)')

plt.tight_layout()
plt.show()

## 💾 Bước 6: Lưu dữ liệu để phân tích tiếp

In [ ]:
# Tạo thư mục data nếu chưa có
import os
os.makedirs('/app/data', exist_ok=True)

# Lưu dữ liệu ra file CSV
customers.to_csv('/app/data/customers.csv', index=False)
products.to_csv('/app/data/products.csv', index=False)
orders.to_csv('/app/data/orders.csv', index=False)
order_details.to_csv('/app/data/order_details.csv', index=False)

print("✅ Đã lưu tất cả dữ liệu vào thư mục /app/data/")
print("📁 Các file đã tạo:")
print("   - customers.csv")
print("   - products.csv")
print("   - orders.csv")
print("   - order_details.csv")

## 🔚 Kết thúc

In [ ]:
# Đóng kết nối database
db.close()
print("🔌 Đã đóng kết nối database")
print("\n✅ Hoàn thành notebook phân tích cơ bản!")
print("➡️ Tiếp theo: Chạy notebook '02_Advanced_Sales_Analysis.ipynb' để phân tích bán hàng chi tiết")